<a href="https://colab.research.google.com/github/deepakatAviSoft/march_assignments/blob/main/08_03_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# installing spark using spark
!pip install pyspark

**>>> Load the Walmart Stock CSV File, have Spark infer the data types.**

In [6]:
# Importing the SparkSession module from the pyspark.sql package
from pyspark.sql import SparkSession

# Creating a SparkSession named "Walmart Analysis" using the SparkSession builder
spark = SparkSession.builder.appName("Walmart Analysis").getOrCreate()

# Reading a CSV file named "walmart_stock.csv" into a DataFrame named stock_df.
# Specifying that the file has a header and inferring the schema automatically.
stock_df = spark.read.csv("walmart_stock.csv", header=True, inferSchema=True)

# Displaying the DataFrame
stock_df


DataFrame[Date: date, Open: double, High: double, Low: double, Close: double, Volume: int, Adj Close: double]

**1. What are the column names?**

In [7]:
# Accessing the columns attribute of the DataFrame stock_df
# This attribute returns a list of column names in the DataFrame.
stock_df.columns


['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

**2. What does the Schema look like?**

In [8]:
# Printing the schema of the DataFrame stock_df
# This provides information about the data types of each column in the DataFrame.
stock_df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



**3. Print out the first 5 columns.**

In [9]:
# Selecting the first five columns of the DataFrame stock_df and displaying the first 5 rows.
# The select() function is used to specify the columns to be selected.
# Here, stock_df.columns[:5] selects the first five column names.
# The show(5) function is used to display the first 5 rows of the resulting DataFrame.
first_5 = stock_df.select(stock_df.columns[:5]).show(5)

# Printing the result of the show() function, which displays the first 5 rows of the DataFrame.
# The show() function returns None, so it doesn't need to be printed.
print(first_5)


+----------+------------------+---------+---------+------------------+
|      Date|              Open|     High|      Low|             Close|
+----------+------------------+---------+---------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18|
+----------+------------------+---------+---------+------------------+
only showing top 5 rows

None


**4. Use describe() to learn about the DataFrame.**

In [10]:
# Generating summary statistics for numerical columns in the DataFrame stock_df and displaying the results.
# The describe() function computes statistics such as count, mean, stddev, min, and max for each numerical column.
# The show() function is used to display the summary statistics in a tabular format.
stock_df.describe().show()


+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

**5. There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar.**


In [11]:
# Importing the format_number function from the pyspark.sql.functions module.
from pyspark.sql.functions import format_number

# Generating summary statistics for the DataFrame stock_df.
descr = stock_df.describe()

# Selecting specific columns from the summary statistics DataFrame and formatting them.
# The format_number function is used to format numerical columns to two decimal places.
# The alias function is used to rename the formatted columns.
formatted = descr.select(
    descr['summary'],
    format_number(descr['Open'].cast('float'), 2).alias('Open'),
    format_number(descr['High'].cast('float'), 2).alias('High'),
    format_number(descr['Low'].cast('float'), 2).alias('Low'),
    format_number(descr['Close'].cast('float'), 2).alias('Close'),
    format_number(descr['Volume'].cast('float'), 2).alias('Volume')
).show()



+-------+--------+--------+--------+--------+-------------+
|summary|    Open|    High|     Low|   Close|       Volume|
+-------+--------+--------+--------+--------+-------------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096.00|
+-------+--------+--------+--------+--------+-------------+



**6. Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day**

In [12]:
# Importing the col function from the pyspark.sql.functions module.
from pyspark.sql.functions import col

# Creating a new column 'HV Ratio' in the DataFrame stock_df.
# This column represents the ratio of High to Volume.
hv_ratio = stock_df.withColumn('HV Ratio', col('High') / col('Volume'))

# Selecting only the 'HV Ratio' column from the DataFrame hv_ratio and displaying it.
hv_ratio.select("HV Ratio").show()


+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



**7. What day had the Peak High in Price?**

In [13]:
# Importing the desc function from the pyspark.sql.functions module.
from pyspark.sql.functions import desc

# Sorting the DataFrame stock_df by the 'High' column in descending order.
# Then, selecting only the 'Date' column.
# The first() function retrieves the first row of the DataFrame after sorting.
# [0] is used to access the first element (Date) of the first row.
Highest_price_day = stock_df.orderBy(desc("High")).select("Date").first()[0]

# Printing the Date of the day with the highest price.
Highest_price_day


datetime.date(2015, 1, 13)

**8. What is the max and min of the Volume column?**

In [14]:
# Importing the max and min functions from the pyspark.sql.functions module.
from pyspark.sql.functions import max, min

# Selecting the maximum and minimum values of the 'Volume' column from the DataFrame stock_df.
# The max() function calculates the maximum value of the 'Volume' column,
# and the min() function calculates the minimum value of the 'Volume' column.
max_min_vol = stock_df.select(max("Volume"), min("Volume"))

# Displaying the maximum and minimum values of the 'Volume' column.
max_min_vol.show()


+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



**9. How many days was the Close lower than 60 dollars?**

In [15]:
# Importing the col function from the pyspark.sql.functions module.
from pyspark.sql.functions import col

# Filtering the DataFrame stock_df to count the number of days where the 'Close' price is less than 60.
# The filter() function is used with the condition col("Close") < 60.
# The count() function calculates the number of rows that satisfy the condition.
days_lower_close_60 = stock_df.filter(col("Close") < 60).count()

# Printing the number of days where the 'Close' price is less than 60.
print(days_lower_close_60)


81


**10. What percentage of the time was the High greater than 80 dollars?**

In [16]:
# Importing the col function from the pyspark.sql.functions module.
from pyspark.sql.functions import col

# Filtering the DataFrame stock_df to count the number of days where the 'High' price is greater than 80.
Higher_price_80 = stock_df.filter(col("High") > 80).count()

# Calculating the total number of days in the DataFrame stock_df.
total = stock_df.count()

# Calculating the percentage of days where the 'High' price is greater than 80.
# The percentage is calculated as (count of days with High > 80) / (total count of days) * 100.
percentage = (Higher_price_80 / total) * 100

# Rounding the percentage value to 6 decimal places and printing it.
print(round(percentage, 6))


9.141494


**11. What is the max High per year**

In [17]:
# Importing the max and year functions from the pyspark.sql.functions module.
from pyspark.sql.functions import max, year

# Creating a new column 'Year' in the DataFrame stock_df to extract the year from the 'Date' column.
# The year() function extracts the year from the 'Date' column.
df_Years = stock_df.withColumn('Year', year(stock_df['Date']))

# Grouping the DataFrame df_Years by the 'Year' column and aggregating the maximum 'High' price for each year.
# The agg() function is used to apply the aggregation function max() to the 'High' column and aliasing it as 'MaxHighAllYears'.
highest_of_all_years = df_Years.groupBy('Year').agg(max('High').alias('MaxHighAllYears'))

# Displaying the DataFrame containing the highest 'High' price for each year.
highest_of_all_years.show()


+----+---------------+
|Year|MaxHighAllYears|
+----+---------------+
|2015|      90.970001|
|2013|      81.370003|
|2014|      88.089996|
|2012|      77.599998|
|2016|      75.190002|
+----+---------------+



**12. What is the Pearson correlation between High and Volume?**

In [18]:
# Importing the corr function from the pyspark.sql.functions module.
from pyspark.sql.functions import corr

# Calculating the correlation between the 'High' and 'Volume' columns in the DataFrame stock_df.
# The corr() function computes the correlation coefficient between two columns.
correlation_high_vol = stock_df.corr('High', 'Volume')

# Printing the correlation coefficient between the 'High' and 'Volume' columns.
correlation_high_vol


-0.3384326061737161

**13.** What is the average Close for each Calendar Month?
In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months.


In [19]:
# Importing the month and avg functions from the pyspark.sql.functions module.
from pyspark.sql.functions import month, avg

# Creating a new column 'Months' in the DataFrame stock_df to extract the month from the 'Date' column.
# The month() function extracts the month from the 'Date' column.
df_month = stock_df.withColumn('Months', month(stock_df['Date']))

# Grouping the DataFrame df_month by the 'Months' column and calculating the average 'Close' price for each month.
# The agg() function is used to apply the aggregation function avg() to the 'Close' column
# and aliasing it as 'Average Closing Price'.
new = df_month.groupby('Months').agg(avg('Close').alias('Average Closing Price'))

# Ordering the resulting DataFrame monthly_closing_price by the 'Months' column.
monthly_closing_price = new.orderBy('Months')

# Displaying the DataFrame containing the average closing price for each month.
monthly_closing_price.show()


+------+---------------------+
|Months|Average Closing Price|
+------+---------------------+
|     1|    71.44801958415842|
|     2|      71.306804443299|
|     3|    71.77794377570092|
|     4|    72.97361900952382|
|     5|    72.30971688679247|
|     6|     72.4953774245283|
|     7|    74.43971943925233|
|     8|    73.02981855454546|
|     9|    72.18411785294116|
|    10|    71.57854545454543|
|    11|     72.1110893069307|
|    12|    72.84792478301885|
+------+---------------------+

